In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import tensorflow as tf
print(tf.__version__)


2025-04-16 22:11:05.621137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744841465.825399      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744841465.883620      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.18.0


In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("benjaminawd/new-york-times-articles-comments-2020")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/new-york-times-articles-comments-2020


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import random

# Load dataset
articles_path = '/kaggle/input/new-york-times-articles-comments-2020/nyt-articles-2020.csv'
df = pd.read_csv(articles_path, usecols=['headline'])
df.dropna(inplace=True)

# Prepare text data
headlines = df['headline'].astype(str).tolist()

# Tokenization
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(headlines)
total_words = 5000

input_sequences = []
for headline in headlines:
    token_list = tokenizer.texts_to_sequences([headline])[0]
    for i in range(1, len(token_list)):
        input_sequences.append(token_list[:i+1])

input_sequences = input_sequences[-100000:]  # cap to 100k
max_sequence_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

X, y = input_sequences[:, :-1], input_sequences[:, -1]

model = Sequential([
    Embedding(total_words, 64, input_length=max_sequence_length-1),
    LSTM(64),
    Dense(64, activation='relu'),
    Dense(total_words, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=30, verbose=1)



# Generate new headlines
def generate_headline(seed_text, next_words=10):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        output_word = tokenizer.index_word.get(predicted[0], "")
        seed_text += " " + output_word
    return seed_text



2025-04-16 20:16:14.481591: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744834574.738392      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744834574.809727      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1744834589.887786      31 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1744834589.888487      31 gpu_device.cc:

Epoch 1/30


I0000 00:00:1744834594.166323      93 cuda_dnn.cc:529] Loaded cuDNN version 90300


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.1202 - loss: 6.6883
Epoch 2/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.1412 - loss: 6.0839
Epoch 3/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.1628 - loss: 5.7786
Epoch 4/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.1686 - loss: 5.5672
Epoch 5/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.1799 - loss: 5.3817
Epoch 6/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.1846 - loss: 5.2282
Epoch 7/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.1943 - loss: 5.0744
Epoch 8/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.1990 - loss: 4.9429
Epoch 9/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - accuracy: 0.2034 - loss: 4.8109
Epoch 10/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.2132 - loss: 4.6939
Epoch 11/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.2199 - loss: 4.5764
Epoch 12/30
3125/3125 ━━━━━━━━

In [10]:
print(generate_headline("Breaking news"))
print(generate_headline("new release"))
print(generate_headline("white house"))
print(generate_headline("wall street"))

Breaking news <OOV> leave the pandemic to <OOV> <OOV> uncertainty and confusion
new release old <OOV> <OOV> of <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>
white house <OOV> the <OOV> of <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>
wall street rallies for a storm <OOV> skies were seven 15 years


Instead of always picking the most probable word (argmax), you now:

✅ sample_with_temperature(preds, temperature)
Adds randomness to predictions by adjusting the temperature.

A lower temperature (e.g., 0.2) makes the model more confident and conservative (picks high-probability words).

A higher temperature (e.g., 1.5) makes it more adventurous and diverse (picks from broader options).

In [11]:
def sample_with_temperature(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-7) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_headline(seed_text, next_words=10, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        preds = model.predict(token_list, verbose=0)[0]
        predicted_index = sample_with_temperature(preds, temperature)
        output_word = tokenizer.index_word.get(predicted_index, "")

        if output_word and output_word != "<OOV>":
            seed_text += " " + output_word
        # Optional: else skip adding <OOV> or unknown word

    return seed_text


In [12]:
print(generate_headline("Breaking news"))
print(generate_headline("new release"))
print(generate_headline("white house"))
print(generate_headline("wall street"))

Breaking news awards ‘where are like covering someone made the same
new release from black police plan with cheese is now
white house has been free of being a secret of
wall street shakes off n y c ’s cities march in


In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Load dataset
articles_path = '/kaggle/input/new-york-times-articles-comments-2020/nyt-articles-2020.csv'
df = pd.read_csv(articles_path, usecols=['headline'])
df.dropna(inplace=True)

# Prepare text data
headlines = df['headline'].astype(str).tolist()

# Tokenization with limited vocabulary
vocab_size = 10000
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(headlines)
total_words = vocab_size

# Create input sequences
input_sequences = []
for headline in headlines:
    token_list = tokenizer.texts_to_sequences([headline])[0]
    for i in range(1, len(token_list)):
        input_sequences.append(token_list[:i+1])

# Limit sequences for memory safety
input_sequences = input_sequences[-100000:]

# Pad sequences
max_sequence_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# Split into input and output
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Build LSTM model
model = Sequential([
    Embedding(total_words, 64, input_length=max_sequence_length-1),
    LSTM(64),
    Dense(64, activation='relu'),
    Dense(total_words, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X, y, epochs=30, verbose=1)

# Temperature-based sampling
def sample_with_temperature(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-7) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Headline generation function
def generate_headline(seed_text, next_words=10, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        preds = model.predict(token_list, verbose=0)[0]
        predicted_index = sample_with_temperature(preds, temperature)
        output_word = tokenizer.index_word.get(predicted_index, "")
        if output_word and output_word != "<OOV>":
            seed_text += " " + output_word
    return seed_text

# Example usage
print(generate_headline("tech stocks", next_words=10, temperature=0.8))


Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.0615 - loss: 7.3515
Epoch 2/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - accuracy: 0.0756 - loss: 6.7725
Epoch 3/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.1040 - loss: 6.4531
Epoch 4/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.1190 - loss: 6.1891
Epoch 5/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.1348 - loss: 5.9574
Epoch 6/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - accuracy: 0.1437 - loss: 5.7511
Epoch 7/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.1549 - loss: 5.5431
Epoch 8/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.1664 - loss: 5.3867
Epoch 9/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - accuracy: 0.1727 - loss: 5.2455
Epoch 10/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.1798 - loss: 5.1022
Epoch 11/30
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - accuracy: 0.1879 - loss: 4.9497
Epoch 12/30
3125/3125 ━━━━━━━━

In [14]:

print(generate_headline("Breaking news", next_words=10, temperature=0.8))
print(generate_headline("new release", next_words=10, temperature=0.8))
print(generate_headline("white house", next_words=10, temperature=0.8))
print(generate_headline("wall street", next_words=10, temperature=0.8))
print(generate_headline("tech stocks", next_words=10, temperature=0.8))

Breaking news five profile tries to fake pandemic us later it’s
new release about safety slave mail in ballots in 2 000 student
white house primary to 900 years after the coronavirus doing voters really
wall street rallies in america south 5 million 2 million weeks from


You doubled the vocabulary, giving the model more words to work with.

This can help capture more nuance and variety in headline generation.

In [ ]:
 New Addition: Word Repetition Check
You introduced a last_word tracker to avoid repeating the same word twice in a row:

In [15]:
def generate_headline(seed_text, next_words=10, temperature=1.0):
    last_word = None
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        preds = model.predict(token_list, verbose=0)[0]
        predicted_index = sample_with_temperature(preds, temperature)
        output_word = tokenizer.index_word.get(predicted_index, "")

        if output_word and output_word != "<OOV>" and output_word != last_word:
            seed_text += " " + output_word
            last_word = output_word

    return seed_text

In [17]:

print(generate_headline("Breaking news", next_words=30, temperature=0.8))
print(generate_headline("new release", next_words=30, temperature=0.8))
print(generate_headline("white house", next_words=30, temperature=0.8))
print(generate_headline("wall street", next_words=30, temperature=0.8))
print(generate_headline("tech stocks", next_words=30, temperature=0.8))

Breaking news terrify in breonna taylor case machine else have to begin four bowl allegation t athletes tries to reopen music dies at 94 thanksgiving bond this medical reds brace for the
new release to not the supreme court rules why did it work when you think it’s like there are going online to haunt our dog promised country seeking live air
white house g o p u s finished head by coronavirus surges republicans really see it what should fall good months in four states study for george floyd death tests and
wall street rallies but the trump administration sees each other king in a conservative wasn’t than me stage from the n to wisconsin that ’ new york stop old filmmaker he t
tech stocks face a new risks of u s to fight away about u s won by and lands it to back caught with week people in the times this


In [18]:
from transformers import pipeline

# Load zero-shot classification pipeline
classifier = pipeline("zero-shot-classification")

# Semantic safety check function
def is_semantically_safe(headline, unsafe_topics=["violence", "politics", "tragedy"]):
    result = classifier(headline, candidate_labels=unsafe_topics)
    top_label = result["labels"][0]
    if result["scores"][0] > 0.5:
        return False
    return True


No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [20]:
def generate_safe_headlines(seed_text, count=5, next_words=10, temperature=1.0):
    headlines = []
    attempts = 0
    max_attempts = count * 10  # avoid infinite loop

    while len(headlines) < count and attempts < max_attempts:
        generated = generate_headline(seed_text, next_words, temperature)
        if is_semantically_safe(generated):
            headlines.append(generated)
        attempts += 1

    return headlines


In [21]:
safe_headlines = generate_safe_headlines("technology trends", count=5, temperature=0.8)
for i, h in enumerate(safe_headlines, 1):
    print(f"{i}. {h}")


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


1. technology trends the vibe club takes back to bases in
2. technology trends the days of the coronavirus study finds also 2nd
3. technology trends the state adjusts you face paper and former prude’s d
4. technology trends from his army account for the birth
5. technology trends a half community of ‘the crown’ is covid 19 question


In [22]:
def generate_safe_headline(seed_text, next_words=10, temperature=1.0):
    max_attempts = 10
    attempts = 0

    while attempts < max_attempts:
        generated = generate_headline(seed_text, next_words, temperature)
        if is_semantically_safe(generated):
            return generated
        attempts += 1

    return "Could not generate a safe headline."


In [25]:
print(generate_safe_headline("Breaking news", next_words=19, temperature=0.7))
print(generate_safe_headline("new release", next_words=19, temperature=0.7))
print(generate_safe_headline("white house", next_words=19, temperature=0.7))
print(generate_safe_headline("wall street", next_words=19, temperature=0.7))
print(generate_safe_headline("Btech stocks", next_words=19, temperature=0.7))

Breaking news russia warns to reopen he goes to the at english weight is coming fever people can’t meet n
new release from drug in a virus is breaking up against my most vulnerable t d to the last
Could not generate a safe headline.
wall street rallies for an algorithm vote looked and what is jobless benefits about door on flash floods
Btech stocks face the u s a simple agenda without months round more another business for voting in the pandemic


In [27]:
model.save("your_model_path.h5")



In [28]:
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


In [29]:
model.save('/kaggle/working/headline_model.h5')



In [30]:
import pickle

with open('/kaggle/working/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


In [31]:
from IPython.display import FileLink

FileLink('/kaggle/working/headline_model.h5')


/kaggle/working/headline_model.h5

In [32]:
from IPython.display import FileLink

# Create download link for the saved tokenizer
FileLink('/kaggle/working/tokenizer.pkl')


/kaggle/working/tokenizer.pkl

In [ ]:
import tensorflow as tf
print(tf.__version__)
